# Logistic (Softmax) Regression from scratch

## 1. Mathematical Understanding

Logistic regression models the probability that an observation belongs to a particular category. 
To generate these probabilities, logistic regression uses the **sigmoid** function. This function maps a real number to a value between 0 and 1.
\begin{equation} 
\sigma(z) = \frac{1}{1+e^{-z}} --- (1)
\end{equation}

If x is the input, $\omega$ is the weight of each input, then weighted sum of the evidence for the class(z):
\begin{equation}
z = \sum_{i=1}^n\omega_ix_i \quad(taking\space x_i = 1)
\end{equation}
\begin{equation}
\Rightarrow z = \vec{\omega} X) --- (2)
\end{equation}
where $\omega$ is a coefficient vector and X is the vector of all the observation

So, if probability of P(y=1|x) = $\hat{y}\space then,\space \hat{y} = \sigma(\vec{\omega} x)$<br>
$\Rightarrow$ P(y=1|x) = $\sigma(\vec{\omega} x)\quad$ --- (3)

For a given x, we say yes if the probability P(y = 1|x) is more than 0.5, and no otherwise. We call 0.5 the decision boundary. So, decision(x) = 1 if P(y=1|x)$\geq$ 0.5, 0 otherwise

First, we randomly allocate values to $\omega$. Then, in each iteration (in the range of epochs) this value is systematically changed to get the final coefficient vector. The change is as follows:<br>
<center>$\omega_{new} = \omega_{old} - \eta X\quad$ ---- (4)</center><br>
$\eta \rightarrow$ Learning rate (usually between 0.0001 to 0.01)<br>

Loss function for this model is L($\hat{y}$, y) implies how much $\hat{y}$ differs from the true y. This is given by:<br>
<center>$L = \frac{-1}{m}(y \log(\hat{y}) + (1 - y) \log(1 - \hat{Y}))\quad$ ----- (5)

Our goal with gradient descent is to find the optimal weights: minimize the loss function we’ve defined for the model. So the goal is to find the set of weights which minimizes the loss function, averaged over all examples:<br>
<center>$\theta_{t+1} = \theta_t - \eta \nabla L(f(x;\theta), y)\quad$---- (6)</center><br>
$\theta\rightarrow$ Parameter (weight/ bias...)<br>
This equation is known as the cross entropy loss function.

For datasets with 3 or more categorical value or label, we use softmax regression, or multinomial regression. Here instead of sigmoid function, we use softmax function.<br>
<center>$\sigma(\mathbf{z})_i = \frac{e^{z_i}}{\sum_{j=1}^{K} e^{z_j}}$</center>

## Our Model From Scratch:

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

In [2]:
# This class will process the given file to execute features and target into X and y numpy array
class DataPreprocessor:
    def __init__(self, filepath):
        self.given_data = pd.read_csv(filepath) # Read the data
        self.X, self.y = self.data_cleaning()

    def data_cleaning(self): # Clean the data i.e. features and target into X and y numpy array
        features = [col for col in self.given_data.columns if 'Feature' in col] # Feature columns are added to features
        self.given_data = self.given_data.dropna() # drop any null value
        data = self.given_data[features].copy()
        data = (data - data.min()) / (data.max() - data.min()) * 9 + 1  # Scale data from 1 to 10
        target = self.given_data['Target'].copy() # Copy the target data to target
        X = np.array(data) # feature array
        y = np.array(target) # target array
        return X, y

In [3]:
# This class is the softmax regression
class SoftmaxRegressionFromScratch:
    def __init__(self, X, y, lr=0.1, epochs=50000): # lr - learning rate,  epochs - no of iterations
        self.X = self.add_intercept(X) # add bias term
        self.y = y
        self.lr = lr
        self.epochs = epochs
        self.num_classes = len(np.unique(y)) # number of classes = no of unique y value
        self.num_features = self.X.shape[1] # number of features = row length of X
        self.weights = np.zeros((self.num_features, self.num_classes)) # weight vector is initialized to 0 with num_feature*num_classes dimension
        self.train() # Train the model

    def add_intercept(self, X): # adds the bias term
        intercept = np.ones((X.shape[0], 1)) # initializing the bias term
        return np.concatenate((intercept, X), axis=1)
    
    def softmax(self, z): # softmax function
        z -= np.max(z, axis=1, keepdims=True)  # For numerical stability
        exp_z = np.exp(z)
        return exp_z / np.sum(exp_z, axis=1, keepdims=True)

    # Model training
    def train(self):
        for _ in range(self.epochs):
            z = np.dot(self.X, self.weights)
            y_hat = self.softmax(z)
            y_one_hot = self.one_hot_encode(self.y) # For more than 2 types of category each of them assigned to boolians
            error = y_one_hot - y_hat
            gradient = np.dot(self.X.T, error) / self.X.shape[0]
            self.weights += self.lr * gradient
            
    def weight(self): # return weights
        return self.weights
    
    def one_hot_encode(self, y): # For more than 2 types of category each of them assigned to boolians
        unique_labels = np.unique(y)
        one_hot = np.zeros((y.shape[0], unique_labels.shape[0])) # Initialized to 0
        for i, label in enumerate(unique_labels):
            one_hot[y == label, i] = 1 # Boolian assignment
        return one_hot

    def predict(self, X): # predict the y value from X
        X = self.add_intercept(X) # Add bias term
        z = np.dot(X, self.weights) # equation 2
        y_hat = self.softmax(z) # equation 3
        return np.argmax(y_hat, axis=1)

In [4]:
class ModelEvaluator:
    @staticmethod
    def model_accuracy(y_true, y_pred):
        correct = np.sum(y_true == y_pred) # correct: correctly predicted y values
        total = len(y_true)
        accuracy = correct / total
        return accuracy

## 2 . (a) Training the scratch model

### [Dataset 1]

In [5]:
# Preprocess Data
data_preprocessor1 = DataPreprocessor('data1_train.csv')
X_train1, y_train1 = data_preprocessor1.X, data_preprocessor1.y

In [6]:
soft_model1 = SoftmaxRegressionFromScratch(X_train1, y_train1) # model initialization using train data

In [7]:
y_pred_train1 = soft_model1.predict(X_train1) # prediction

In [8]:
score = ModelEvaluator.model_accuracy(y_train1, y_pred_train1) # evaluation
print(f'Accuracy: {score*100}%')

Accuracy: 97.5%


### [Dataset 2]

In [9]:
# Preprocess Data 
data_preprocessor2 = DataPreprocessor('data2_train.csv')
X_train2, y_train2 = data_preprocessor2.X, data_preprocessor2.y

In [10]:
soft_model2 = SoftmaxRegressionFromScratch(X_train2, y_train2)

In [11]:
y_pred_train2 = soft_model2.predict(X_train2)

In [12]:
score = ModelEvaluator.model_accuracy(y_train2, y_pred_train2)
print(f'Accuracy: {score*100}%')

Accuracy: 98.875%


## 2. (b) Testing the scratch model

### [Dataset 1]

In [13]:
data_preprocessor_test1 = DataPreprocessor('data1_test.csv')
X_test1, y_test1 = data_preprocessor_test1.X, data_preprocessor_test1.y

In [14]:
# Predict on the test set
y_pred_test1 = soft_model1.predict(X_test1)
print("Predictions on test set:", np.bincount(y_pred_test1))

Predictions on test set: [70 70 60]


In [15]:
score = ModelEvaluator.model_accuracy(y_test1, y_pred_test1)
print(f'Accuracy: {score*100}%')

Accuracy: 96.5%


### [Dataset 2]

In [16]:
data_preprocessor_test2 = DataPreprocessor('data2_test.csv')
X_test2, y_test2 = data_preprocessor_test2.X, data_preprocessor_test2.y

In [17]:
# Predict on the test set
y_pred_test2 = soft_model2.predict(X_test2)
print("Predictions on test set:", np.bincount(y_pred_test2))

Predictions on test set: [ 93 107]


In [18]:
score = ModelEvaluator.model_accuracy(y_test2, y_pred_test2)
print(f'Accuracy: {score*100}%')

Accuracy: 94.0%


## 3. Hyperparameter Tuning

In [19]:
# two hyperparameter: learning rate, epochs
class GridSearchSoftmaxRegression(SoftmaxRegressionFromScratch):
    def __init__(self, X, y, lr=0.1, epochs=50000):
        super().__init__(X, y, lr, epochs)

    def grid_search(self, X, y, param_grid):
        best_params = None
        best_score = 0

        for lr in param_grid['lr']: # Iterating over given learning rates
            for epochs in param_grid['epochs']: # Iterating over given epochs
                self.lr = lr
                self.epochs = epochs
                self.weights = np.zeros((self.num_features, self.num_classes))  # Reset weights
                self.train()
                score = self.evaluate(X, y) #evaluate score to check performance

                if score > best_score: # taking the best score
                    best_score = score
                    best_params = {'lr': lr, 'epochs': epochs}

        return best_params, best_score # got the best parameter and score at the end the nested loop

    def evaluate(self, X, y):
        y_pred = self.predict(X)
        accuracy = ModelEvaluator.model_accuracy(y, y_pred)
        return accuracy

In [20]:
# Define the hyperparameter grid: lr - learning rate
param_grid = {
    'lr': [0.01, 0.05, 0.1, 0.5],
    'epochs': [1000, 5000, 10000]
}

### On Dataset 1

In [21]:
# search for the best parameter and value
grid_search_model = GridSearchSoftmaxRegression(X_train1, y_train1)
best_params1, best_score1 = grid_search_model.grid_search(X_train1, y_train1, param_grid)
print(f"Best Parameters: {best_params1}, Best Score: {best_score1*100}%")

Best Parameters: {'lr': 0.5, 'epochs': 5000}, Best Score: 97.625%


In [22]:
# Initialize a model using the best value to compare accuracy
best_model1 = GridSearchSoftmaxRegression(X_train1, y_train1, lr=best_params1['lr'], epochs=best_params1['epochs'])

In [23]:
tuned_y_pred1 = best_model1.predict(X_test1)
print("Predictions on test set:", np.bincount(tuned_y_pred1))
score = ModelEvaluator.model_accuracy(y_test1, tuned_y_pred1)
print(f'Accuracy: {score*100}%')

Predictions on test set: [70 70 60]
Accuracy: 96.5%


### On Dataset 2

In [24]:
grid_search_model = GridSearchSoftmaxRegression(X_train2, y_train2)
best_params2, best_score2 = grid_search_model.grid_search(X_train2, y_train2, param_grid)
print(f"Best Parameters: {best_params2}, Best Score: {best_score2*100}%")

Best Parameters: {'lr': 0.1, 'epochs': 10000}, Best Score: 99.0%


In [25]:
best_model2 = GridSearchSoftmaxRegression(X_train2, y_train2, lr=best_params2['lr'], epochs=best_params2['epochs'])

In [26]:
tuned_y_pred2 = best_model2.predict(X_test2)
print("Predictions on test set:", np.bincount(tuned_y_pred2))
score = ModelEvaluator.model_accuracy(y_test2, tuned_y_pred2)
print(f'Accuracy: {score*100}%')

Predictions on test set: [ 94 106]
Accuracy: 93.5%


## 4. Comparision with Scikit-Learn

In [27]:
from sklearn.linear_model import LogisticRegression # Logistic regression from scikit learn

class SkLearnLogisticRegression:
    def __init__(self):
        self.model = LogisticRegression() # Define the model
    
    def train(self, X_train, y_train): # Train the model using model.fit() function 
        self.model.fit(X_train, y_train)
    
    def predict(self, X_test): # Predict the model output using model.predict() function 
        return self.model.predict(X_test)
    
    def evaluate_accuracy(self, y_true, y_pred): # evaluate the model
        self.accuracy = accuracy_score(y_true, y_pred)
        return self.accuracy      

In [28]:
sklearn_model = SkLearnLogisticRegression() # Initialize the model

### - On Dataset 1 (train & test)

In [29]:
sklearn_model.train(X_train1, y_train1) # Train the model using train1 dataset

In [30]:
y_sk_pred1 = sklearn_model.predict(X_test1) # predict y/target value

In [31]:
# Print the accuracy score
score = sklearn_model.evaluate_accuracy(y_test1, y_sk_pred1)
print(f'Accuracy: {score*100}%')

Accuracy: 96.5%


### - On Dataset 2 (train & test)

In [32]:
# Same as previous but with train2 dataset
sklearn_model.train(X_train2, y_train2)

In [33]:
y_sk_pred2 = sklearn_model.predict(X_test2)
score = sklearn_model.evaluate_accuracy(y_test2, y_sk_pred2)
print(f'Accuracy: {score*100}%')

Accuracy: 94.5%
